In [0]:
print ("Hello there again")

In [0]:
%sql
use catalog firstcatalog;
use schema firstschema;
select current_catalog(), current_schema()

# Describe the schema

In [0]:
%sql
DESCRIBE SCHEMA EXTENDED firstcatalog.firstschema

# Show tables

In [0]:
%sql
show tables

#Show Volumes

In [0]:
%sql
show volumes;

#Listing files/data in Volumes
Use the path provided in the Unity Catalog: /Volumes/catalog_name/schema_name/volume_name/

In [0]:
catalog = "firstcatalog"
schema = "firstschema"
volume = "firstvolume";
spark.sql(f"LIST'/Volumes/{catalog}/{schema}/{volume}'").display()

# Query the file directly as CSV
This syntax is specific to Spark SQL and allows you to read files directly without explicit loading them into a table. You specify the file format and enclose the file path into backticks
SELECT * from file_format.`/path/to/file` 

In [0]:
spark.sql(f'''
          SELECT * from csv.`/Volumes/{catalog}/{schema}/{volume}`
          ''').display()

# Query the file directly as Text

In [0]:
spark.sql(f'''
          SELECT * from text.`/Volumes/{catalog}/{schema}/{volume}`
          ''').display()

## Read the CSV file into tabular form
by using the read_files table-valued function

In [0]:
%sql
SELECT * FROM read_files('/Volumes/firstcatalog/firstschema/firstvolume',
format => 'csv',
header => true,
inferSchema => true
)

## Create a table from the file

In [0]:
%sql
DROP TABLE IF EXISTS online_retail;
DROP TABLE IF EXISTS firstcatalog.firstschema.online_retail; 

CREATE TABLE firstcatalog.firstschema.online_retail AS
SELECT InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country
FROM read_files('/Volumes/firstcatalog/firstschema/firstvolume',
format => 'csv',
header => true,
inferSchema => true
);

SELECT * FROM firstcatalog.firstschema.online_retail; 

## Use Python to create a Delta table

In [0]:
# Create the dataframe
sdf = (spark
.read
.format("csv")
.option("header", "true")
.option("inferSchema", "true")
.load(f'/Volumes/{catalog}/{schema}/{volume}')
)

(sdf
 .write
 .mode("overwrite")
 .format("delta")
 .saveAsTable(f'{catalog}.{schema}.online_retail_py')
)


## Query the table using Python

In [0]:
spark.read.table(f'{catalog}.{schema}.online_retail_py').display()

## List tables using Python 

In [0]:
spark.catalog.listTables(f'{catalog}.{schema}')

## Describe Detail

In [0]:
%sql
DESCRIBE DETAIL firstcatalog.firstschema.online_retail;

## Describe Extended

In [0]:
%sql
DESCRIBE EXTENDED firstcatalog.firstschema.online_retail;

## Describe History

In [0]:
%sql
DESCRIBE HISTORY firstcatalog.firstschema.online_retail; 

In [0]:
%sql
SELECT * FROM firstcatalog.firstschema.online_retail
WHERE InvoiceNo='536365'
AND StockCode='85123A';

## Update the table

In [0]:
%sql
UPDATE firstcatalog.firstschema.online_retail
SET Quantity=10
WHERE InvoiceNo='536365'
AND StockCode='85123A';

## Time travel


In [0]:
%sql
--VERSION AS OF 1 or @v1
SELECT * FROM firstcatalog.firstschema.online_retail@v0
WHERE InvoiceNo='536365'
AND StockCode='85123A';

## Drop Tables

In [0]:
%sql
DROP TABLE IF EXISTS firstcatalog.firstschema.online_retail; 
DROP TABLE IF EXISTS firstcatalog.firstschema.online_retail_py; 

## Practice Python

In [0]:
##See what files we have in the schema
#spark.sql(f"LIST '/Volumes/firstcatalog/firstschema/firstvolume'").display()
##Read the text file
#spark.sql('''
#          SELECT * FROM text.`/Volumes/firstcatalog/firstschema/#firstvolume`
#          ''').display()
##Read the CSV file
#spark.sql("SELECT * from csv.`/Volumes/firstcatalog/firstschema/firstvolume`").display()

##Create a spark dataframe
#sdf = (spark
#.read
#.format("csv")
#.option("inferSchema", "true")
#.option("header", "true")
#.load("/Volumes/firstcatalog/firstschema/firstvolume"))

#sdf.show()

##Create a table from dataframe
#(sdf
#.write
#.mode("overwrite")
#.format("delta")
#.saveAsTable("firstcatalog.firstschema.online_retail_py"))

#spark.read.table("firstcatalog.firstschema.online_retail_py").display()

spark.catalog.listTables("firstcatalog.firstschema")


## Practice SQL


In [0]:
%sql
--DESCRIBE DETAIL firstcatalog.firstschema.online_retail_py
--DESCRIBE EXTENDED firstcatalog.firstschema.online_retail_py 
--DESCRIBE HISTORY firstcatalog.firstschema.online_retail_py

--SELECT * FROM read_files ('/Volumes/firstcatalog/firstschema/firstvolume',
--format => "csv",
--header => true,
--inferSchema => true)

--DROP TABLE IF EXISTS firstcatalog.firstschema.online_retail_sql

--CREATE TABLE firstcatalog.firstschema.online_retail_sql AS
--SELECT * FROM read_files('/Volumes/firstcatalog/firstschema/firstvolume',
--format => 'csv',
--header => true,
--inferSchema => true)

--SELECT * FROM firstcatalog.firstschema.online_retail_sql
--UPDATE firstcatalog.firstschema.online_retail_py
--SET Quantity=10
--WHERE InvoiceNo='536365'

SELECT * FROM firstcatalog.firstschema.online_retail_sql@v1